# Backtesting the HMM Model

Signals are identifed by the hmm model and imported into the system. 

    When signal = 1, we enter the mean reversion position.    
    When signal = 0, we enter the momentum position. 


Mean Reversion Strategy:

     1.If return in the past 21 days > 0.02, we sell     
     2.If return in the past 21 days < -0.02, we buy     
     3.We close the position every 10 days 


Momentum Strategy: 

    1.If return in the past 6 months > 0.05, we sell
    2.If return in the past 6 months < -0.05, we buy
    3.We close the position every month 
    
    
    


In [57]:
l = [1, 2, 3]
l[1]

2

In [32]:
from __future__ import (absolute_import, division, print_function,
                        unicode_literals)

import datetime  # For datetime objects
import os.path  # To manage paths
import sys  # To find out the script name (in argv[0])

# Import the backtrader platform
import backtrader as bt

import pandas as pd
import datetime

In [83]:

class RSI_Oversold(bt.Indicator):
    lines = ('oversold',)
    
    params = (
        ('level', 30),    
    )

    def __init__(self):
        self.lines.oversold = bt.indicators.RSI(self.data) < self.p.level

        
class DummyInd(bt.Indicator):
    lines = ('dummyline',)

    params = (('value', 5),)

    def __init__(self):
        self.lines.dummyline = bt.Max(0.0, self.params.value)

        
# Create a Stratey
class BackTradingStrategy(bt.Strategy):
    params = (
        ('period', 128),
        ("max_days_in_trade", 10)
    )

    def log(self, txt, txt2, txt3, dt=None):
        ''' Logging function for this strategy'''
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s, %s, %s' % (dt.isoformat(), txt, txt2, txt3))
        
        
    def read_signals(self):
        signal_data = pd.read_csv("hmm_regime_signals.csv")
        signal_data["datetime"] = signal_data["datetime"].apply(lambda x: datetime.datetime.strptime(x,"%m/%d/%Y"))
        signal_data = signal_data.set_index('datetime')
        
        return signal_data
        

    def __init__(self):
        # Keep a reference to the "close" line in the data[0] dataseries
        # How to access individual etf
        self.xlb_close = self.datas[symbols.index("XLB")].close
        self.xle_close = self.datas[symbols.index("XLE")].close
        self.xlf_close = self.datas[symbols.index("XLF")].close
        self.xli_close = self.datas[symbols.index("XLI")].close
        self.xlk_close = self.datas[symbols.index("XLK")].close
        self.xlp_close = self.datas[symbols.index("XLP")].close
        self.xlu_close = self.datas[symbols.index("XLU")].close
        self.xlv_close = self.datas[symbols.index("XLV")].close
        self.xly_close = self.datas[symbols.index("XLY")].close
        
        self.datadate = self.datas[0].datetime.date
        self.days_in_trade = 0
        self.max_days_in_trade_mr = 10
        self.max_days_in_trade_mo = 21
        self.num_mr = 21
        self.num_mo = 21*6
        self.regime_state = 0
        self.mrr = 0.02
        self.mor = 0.05
        self.snum = 3
        self.signal_data = self.read_signals()
        # keep this 
        self.sma = bt.indicators.SimpleMovingAverage(
            self.datas[0], period=self.params.period)        
        self.symbols = ['XLB', 'XLE', 'XLF', 'XLI', 'XLK', 'XLP', 'XLU', 'XLV', 'XLY']
        self.regimes = ['momentum', 'mean reversion']
        
        
    def stock_return(self, n):
        """calculate return in the past n days """        
        r_xlb = (self.xlb_close[0] - self.xlb_close[-n])/self.xlb_close[-n]
        r_xle = (self.xle_close[0] - self.xle_close[-n])/self.xle_close[-n]
        r_xlf = (self.xlf_close[0] - self.xlf_close[-n])/self.xlf_close[-n]
        r_xli = (self.xli_close[0] - self.xli_close[-n])/self.xli_close[-n]
        r_xlk = (self.xlk_close[0] - self.xlk_close[-n])/self.xlk_close[-n]
        r_xlp = (self.xlp_close[0] - self.xlp_close[-n])/self.xlp_close[-n]
        r_xlu = (self.xlu_close[0] - self.xlu_close[-n])/self.xlu_close[-n]
        r_xlv = (self.xlv_close[0] - self.xlv_close[-n])/self.xlv_close[-n]
        r_xly = (self.xly_close[0] - self.xly_close[-n])/self.xly_close[-n]
        r_list = [r_xlb, r_xle, r_xlf, r_xli, r_xlk, r_xlp, r_xlu, r_xlv, r_xly]        
        
        df = pd.DataFrame(r_list, index = self.symbols, columns = ['return'])
        
        return df
    
    
    
    def sort_return(self, n):
        """sort return in the past n days in ascending order 
           return the bottom and top 3 return stocks """
        
        df = self.stock_return(n)
        df = df.sort_values(by=['return'])
                
        stock_list = [df.index[0:3].tolist(),df.index[6:].tolist()]
        
        return stock_list
    
    
    
    def check_position(self):
        flag = False
        names = []
        for symbol in self.symbols:
            s_size = self.getposition(data=self.datas[symbols.index(symbol)]).size
            if  s_size != 0:
                flag = True
                names.append(symbol)
        return flag, names
            
                
    

    def next(self):
        # Simply log the closing price of the series from the reference               
        
        dt = self.datadate(0).strftime('%Y-%m-%d')
        
        signal = self.signal_data['state'].loc[dt]
        
        position, names = self.check_position()
        
        self.log('regime = %s' % self.regimes[signal], '      HAS Position at %s' % " ".join(names[:3]), ' and  at  %s' %  " ".join(names[3:]))

        
        
        # Check if we are in the market
        if not position:
            # if we do not have a position
            
            if signal == 1:
                """mean reversion"""
                
                stock_list = self.sort_return(self.num_mr)
                # print('stocklist = ', stock_list)
                
                self.log('regime = %s' % self.regimes[signal], 'CREATE BUY  for %s' % " ".join(stock_list[0]), ' SELL for %s' %  " ".join(stock_list[0])) 

                # Buy bottom snum stocks
                for i in range(self.snum):
                    self.order = self.buy(data = self.datas[symbols.index(stock_list[0][i])])
                    
                # Sell top snum stocks    
                for i in range(self.snum):
                    self.order = self.sell(data = self.datas[symbols.index(stock_list[1][i])])

                self.days_in_trade = 1
                
                self.regime_state = 1
                    
                    
         
            
            else:
                """momentum"""  
                
                stock_list = self.sort_return(self.num_mo)
                # print('stocklist = ', stock_list)
                
                self.log('regime = %s' % self.regimes[signal], '      CREATE SELL for %s' % " ".join(stock_list[0]), ' BUY  for %s' %  " ".join(stock_list[0]))

                # Sell bottom snum stocks
                for i in range(self.snum):
                    self.order = self.sell(data = self.datas[symbols.index(stock_list[0][i])])
                    
                # Buy top snum stocks    
                for i in range(self.snum):
                    self.order = self.buy(data = self.datas[symbols.index(stock_list[1][i])])
                    
                self.days_in_trade = 1
                
                self.regime_state = 0
                    
            
                
            

        else:
            # if we have a position
            
            self.days_in_trade += 1
            
            """mean reversion"""
            if (self.regime_state == 1):
            
                # close the position if regime shift or after holding it for max_mr days.
                if signal == 0 or self.days_in_trade == self.max_days_in_trade_mr:
                    num = len(names)
                    for i in range(num):
                        self.close(data=self.datas[symbols.index(names[i])])
                    
                    self.days_in_trade = 0                    
                    self.regime_state = 0 
                    
            
            """momentum"""
            if (self.regime_state == 0):
                
                # close the position if regime shift or after holding it for max_mo days.
                if signal == 1 or self.days_in_trade == self.max_days_in_trade_mo:
                    num = len(names)
                    for i in range(num):
                        self.close(data=self.datas[symbols.index(names[i])])

                    self.days_in_trade = 0                        
                    self.regime_state = 1
                 
       


In [84]:


if __name__ == '__main__':
    # Create a cerebro entity
    cerebro = bt.Cerebro()   

    symbols = ['XLB', 'XLE', 'XLF', 'XLI', 'XLK', 'XLP', 'XLU', 'XLV', 'XLY']
    for symbol in symbols:
        datapath = os.path.join('../../../datas/%s.csv' % symbol)

        # Create a Data Feed
        data = bt.feeds.YahooFinanceCSVData(
            dataname=datapath,
            # Do not pass values before this date
            fromdate=datetime.datetime(2013, 2, 11),
            # Do not pass values before this date
            todate=datetime.datetime(2018, 2, 9),
            # Do not pass values after this date
            reverse=False)

        # Add the Data Feed to Cerebro
        cerebro.adddata(data, name=symbol)


    # Add a strategy
    cerebro.addstrategy(BackTradingStrategy) #, symbols= symbols)
    
    # Set our desired cash start
    cerebro.broker.setcash(100000.0)
    
    

    # Write output
    cerebro.addwriter(bt.WriterFile, out='0402-ETFbacktesting.csv',csv=True)
    
    # Print out the starting conditions
    print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

    # Run over everything
    cerebro.run()

    # Print out the final result
    print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())

Starting Portfolio Value: 100000.00
2013-08-13, regime = momentum,       HAS Position at ,  and  at  
2013-08-13, regime = momentum,       CREATE SELL for XLE XLB XLU,  BUY  for XLE XLB XLU
2013-08-14, regime = momentum,       HAS Position at XLB XLE XLF,  and  at  XLU XLV XLY
2013-08-15, regime = momentum,       HAS Position at XLB XLE XLF,  and  at  XLU XLV XLY
2013-08-16, regime = momentum,       HAS Position at XLB XLE XLF,  and  at  XLU XLV XLY
2013-08-19, regime = momentum,       HAS Position at XLB XLE XLF,  and  at  XLU XLV XLY
2013-08-20, regime = momentum,       HAS Position at XLB XLE XLF,  and  at  XLU XLV XLY
2013-08-21, regime = momentum,       HAS Position at XLB XLE XLF,  and  at  XLU XLV XLY
2013-08-22, regime = momentum,       HAS Position at XLB XLE XLF,  and  at  XLU XLV XLY
2013-08-23, regime = momentum,       HAS Position at XLB XLE XLF,  and  at  XLU XLV XLY
2013-08-26, regime = momentum,       HAS Position at XLB XLE XLF,  and  at  XLU XLV XLY
2013-08-27, regime

2014-01-08, regime = momentum,       HAS Position at XLE XLI XLP,  and  at  XLU XLV XLY
2014-01-09, regime = momentum,       HAS Position at XLE XLI XLP,  and  at  XLU XLV XLY
2014-01-10, regime = momentum,       HAS Position at XLE XLI XLP,  and  at  XLU XLV XLY
2014-01-13, regime = momentum,       HAS Position at ,  and  at  
2014-01-13, regime = momentum,       CREATE SELL for XLU XLP XLE,  BUY  for XLU XLP XLE
2014-01-14, regime = momentum,       HAS Position at XLB XLE XLI,  and  at  XLP XLU XLV
2014-01-15, regime = momentum,       HAS Position at XLB XLE XLI,  and  at  XLP XLU XLV
2014-01-16, regime = momentum,       HAS Position at XLB XLE XLI,  and  at  XLP XLU XLV
2014-01-17, regime = momentum,       HAS Position at XLB XLE XLI,  and  at  XLP XLU XLV
2014-01-21, regime = momentum,       HAS Position at XLB XLE XLI,  and  at  XLP XLU XLV
2014-01-22, regime = momentum,       HAS Position at XLB XLE XLI,  and  at  XLP XLU XLV
2014-01-23, regime = momentum,       HAS Position at X

2014-05-30, regime = momentum,       HAS Position at XLB XLF XLP,  and  at  XLU XLV XLY
2014-06-02, regime = momentum,       HAS Position at XLB XLF XLP,  and  at  XLU XLV XLY
2014-06-03, regime = momentum,       HAS Position at XLB XLF XLP,  and  at  XLU XLV XLY
2014-06-04, regime = momentum,       HAS Position at XLB XLF XLP,  and  at  XLU XLV XLY
2014-06-05, regime = momentum,       HAS Position at XLB XLF XLP,  and  at  XLU XLV XLY
2014-06-06, regime = momentum,       HAS Position at XLB XLF XLP,  and  at  XLU XLV XLY
2014-06-09, regime = momentum,       HAS Position at XLB XLF XLP,  and  at  XLU XLV XLY
2014-06-10, regime = momentum,       HAS Position at XLB XLF XLP,  and  at  XLU XLV XLY
2014-06-11, regime = momentum,       HAS Position at XLB XLF XLP,  and  at  XLU XLV XLY
2014-06-12, regime = momentum,       HAS Position at XLB XLF XLP,  and  at  XLU XLV XLY
2014-06-13, regime = momentum,       HAS Position at ,  and  at  
2014-06-13, regime = momentum,       CREATE SELL for X

2014-10-21, regime = momentum,       HAS Position at XLB XLE XLI,  and  at  XLK XLP XLV
2014-10-22, regime = momentum,       HAS Position at XLB XLE XLI,  and  at  XLK XLP XLV
2014-10-23, regime = momentum,       HAS Position at XLB XLE XLI,  and  at  XLK XLP XLV
2014-10-24, regime = momentum,       HAS Position at XLB XLE XLI,  and  at  XLK XLP XLV
2014-10-27, regime = momentum,       HAS Position at XLB XLE XLI,  and  at  XLK XLP XLV
2014-10-28, regime = momentum,       HAS Position at XLB XLE XLI,  and  at  XLK XLP XLV
2014-10-29, regime = momentum,       HAS Position at XLB XLE XLI,  and  at  XLK XLP XLV
2014-10-30, regime = momentum,       HAS Position at XLB XLE XLI,  and  at  XLK XLP XLV
2014-10-31, regime = momentum,       HAS Position at XLB XLE XLI,  and  at  XLK XLP XLV
2014-11-03, regime = momentum,       HAS Position at XLB XLE XLI,  and  at  XLK XLP XLV
2014-11-04, regime = momentum,       HAS Position at XLB XLE XLI,  and  at  XLK XLP XLV
2014-11-05, regime = momentum,  

2015-03-11, regime = momentum,       HAS Position at ,  and  at  
2015-03-11, regime = momentum,       CREATE SELL for XLE XLB XLU,  BUY  for XLE XLB XLU
2015-03-12, regime = momentum,       HAS Position at XLB XLE XLP,  and  at  XLU XLV XLY
2015-03-13, regime = momentum,       HAS Position at XLB XLE XLP,  and  at  XLU XLV XLY
2015-03-16, regime = momentum,       HAS Position at XLB XLE XLP,  and  at  XLU XLV XLY
2015-03-17, regime = momentum,       HAS Position at XLB XLE XLP,  and  at  XLU XLV XLY
2015-03-18, regime = momentum,       HAS Position at XLB XLE XLP,  and  at  XLU XLV XLY
2015-03-19, regime = momentum,       HAS Position at XLB XLE XLP,  and  at  XLU XLV XLY
2015-03-20, regime = momentum,       HAS Position at XLB XLE XLP,  and  at  XLU XLV XLY
2015-03-23, regime = momentum,       HAS Position at XLB XLE XLP,  and  at  XLU XLV XLY
2015-03-24, regime = momentum,       HAS Position at XLB XLE XLP,  and  at  XLU XLV XLY
2015-03-25, regime = momentum,       HAS Position at X

...

2016-04-25, regime = momentum,       HAS Position at ,  and  at  
2016-04-25, regime = momentum,       CREATE SELL for XLE XLF XLY,  BUY  for XLE XLF XLY
2016-04-26, regime = momentum,       HAS Position at XLB XLE XLF,  and  at  XLU XLV XLY
2016-04-27, regime = momentum,       HAS Position at XLB XLE XLF,  and  at  XLU XLV XLY
2016-04-28, regime = momentum,       HAS Position at XLB XLE XLF,  and  at  XLU XLV XLY
2016-04-29, regime = momentum,       HAS Position at XLB XLE XLF,  and  at  XLU XLV XLY
2016-05-02, regime = momentum,       HAS Position at XLB XLE XLF,  and  at  XLU XLV XLY
2016-05-03, regime = momentum,       HAS Position at XLB XLE XLF,  and  at  XLU XLV XLY
2016-05-04, regime = momentum,       HAS Position at XLB XLE XLF,  and  at  XLU XLV XLY
2016-05-05, regime = momentum,       HAS Position at XLB XLE XLF,  and  at  XLU XLV XLY
2016-05-06, regime = momentum,       HAS Position at XLB XLE XLF,  and  at  XLU XLV XLY
2016-05-09, regime = momentum,       HAS Position at X

2016-09-02, regime = momentum,       HAS Position at XLB XLE XLF,  and  at  XLP XLU XLY
2016-09-06, regime = momentum,       HAS Position at XLB XLE XLF,  and  at  XLP XLU XLY
2016-09-07, regime = momentum,       HAS Position at XLB XLE XLF,  and  at  XLP XLU XLY
2016-09-08, regime = momentum,       HAS Position at XLB XLE XLF,  and  at  XLP XLU XLY
2016-09-09, regime = momentum,       HAS Position at XLB XLE XLF,  and  at  XLP XLU XLY
2016-09-12, regime = momentum,       HAS Position at XLB XLE XLF,  and  at  XLP XLU XLY
2016-09-13, regime = momentum,       HAS Position at XLB XLE XLF,  and  at  XLP XLU XLY
2016-09-14, regime = momentum,       HAS Position at XLB XLE XLF,  and  at  XLP XLU XLY
2016-09-15, regime = momentum,       HAS Position at XLB XLE XLF,  and  at  XLP XLU XLY
2016-09-16, regime = momentum,       HAS Position at XLB XLE XLF,  and  at  XLP XLU XLY
2016-09-19, regime = mean reversion,       HAS Position at XLB XLE XLF,  and  at  XLP XLU XLY
2016-09-20, regime = momen

2017-01-18, regime = momentum,       CREATE SELL for XLP XLU XLV,  BUY  for XLP XLU XLV
2017-01-19, regime = momentum,       HAS Position at XLE XLF XLK,  and  at  XLP XLU XLV
2017-01-20, regime = momentum,       HAS Position at XLE XLF XLK,  and  at  XLP XLU XLV
2017-01-23, regime = momentum,       HAS Position at XLE XLF XLK,  and  at  XLP XLU XLV
2017-01-24, regime = momentum,       HAS Position at XLE XLF XLK,  and  at  XLP XLU XLV
2017-01-25, regime = momentum,       HAS Position at XLE XLF XLK,  and  at  XLP XLU XLV
2017-01-26, regime = momentum,       HAS Position at XLE XLF XLK,  and  at  XLP XLU XLV
2017-01-27, regime = momentum,       HAS Position at XLE XLF XLK,  and  at  XLP XLU XLV
2017-01-30, regime = momentum,       HAS Position at XLE XLF XLK,  and  at  XLP XLU XLV
2017-01-31, regime = momentum,       HAS Position at XLE XLF XLK,  and  at  XLP XLU XLV
2017-02-01, regime = momentum,       HAS Position at XLE XLF XLK,  and  at  XLP XLU XLV
2017-02-02, regime = momentum,  

2017-06-09, regime = mean reversion,       HAS Position at ,  and  at  
2017-06-09, regime = mean reversion, CREATE BUY  for XLE XLY XLK,  SELL for XLE XLY XLK
2017-06-12, regime = momentum,       HAS Position at XLB XLE XLK,  and  at  XLP XLU XLY
2017-06-13, regime = momentum,       HAS Position at ,  and  at  
2017-06-13, regime = momentum,       CREATE SELL for XLE XLF XLB,  BUY  for XLE XLF XLB
2017-06-14, regime = momentum,       HAS Position at XLB XLE XLF,  and  at  XLK XLU XLV
2017-06-15, regime = momentum,       HAS Position at XLB XLE XLF,  and  at  XLK XLU XLV
2017-06-16, regime = momentum,       HAS Position at XLB XLE XLF,  and  at  XLK XLU XLV
2017-06-19, regime = momentum,       HAS Position at XLB XLE XLF,  and  at  XLK XLU XLV
2017-06-20, regime = momentum,       HAS Position at XLB XLE XLF,  and  at  XLK XLU XLV
2017-06-21, regime = momentum,       HAS Position at XLB XLE XLF,  and  at  XLK XLU XLV
2017-06-22, regime = momentum,       HAS Position at XLB XLE XLF,  and

2017-10-27, regime = mean reversion,       HAS Position at XLB XLE XLF,  and  at  XLK XLP XLY
2017-10-30, regime = momentum,       HAS Position at ,  and  at  
2017-10-30, regime = momentum,       CREATE SELL for XLP XLY XLE,  BUY  for XLP XLY XLE
2017-10-31, regime = momentum,       HAS Position at XLB XLE XLF,  and  at  XLK XLP XLY
2017-11-01, regime = momentum,       HAS Position at XLB XLE XLF,  and  at  XLK XLP XLY
2017-11-02, regime = momentum,       HAS Position at XLB XLE XLF,  and  at  XLK XLP XLY
2017-11-03, regime = momentum,       HAS Position at XLB XLE XLF,  and  at  XLK XLP XLY
2017-11-06, regime = momentum,       HAS Position at XLB XLE XLF,  and  at  XLK XLP XLY
2017-11-07, regime = momentum,       HAS Position at XLB XLE XLF,  and  at  XLK XLP XLY
2017-11-08, regime = momentum,       HAS Position at XLB XLE XLF,  and  at  XLK XLP XLY
2017-11-09, regime = momentum,       HAS Position at XLB XLE XLF,  and  at  XLK XLP XLY
2017-11-10, regime = momentum,       HAS Positio

In [ ]:
# calculate pnl using excel data
import pandas as pd
import matplotlib.pyplot as plt

#price = pd.read_csv('XLB.csv')
data = pd.read_csv('0402-ETFbacktesting.csv').fillna(value = 0)
data = data[:1260]

df = data[[symbols]].copy()
#df['datetime'] = 
n = len(df)

df['cumulative_pnl'] = df['pnlplus']

cpnl = 0.
count_plus = 0
count_minus = 0

for i in range(n):
    plus = df['pnlplus'].iloc[i]
    minus = df['pnlminus'].iloc[i]
    cpnl = cpnl + plus + minus
    df['cumulative_pnl'].iloc[i] = cpnl
    
    if (plus > 0.):
        count_plus +=1
    if (minus < 0.):
        count_minus +=1

    

In [ ]:
plus_percent = 1.0*count_plus/(count_plus*1.0+count_minus*1.0) 
print('percentage of trading that make positive profits = ', plus_percent)

In [ ]:
# convert string to datetime
df["datetime"] = df["datetime"].apply(lambda x: datetime.strptime(x,"%Y-%m-%d %H:%M:%S.%f"))

# plot PNL figures 
plt.figure(figsize=(16, 4))
plt.plot(df['datetime'], df['cumulative_pnl'] ,'.') 
plt.xlabel("Time")
plt.ylabel("cumulative pnl")
plt.title("Daily PNL")
plt.show()